In [1]:
import pandas as pd

In [2]:
osm_routepoints = pd.read_csv("../data/osm_routepoints.csv", dtype=str)
for c in osm_routepoints.columns:
    new_col = "osm_" + c
    osm_routepoints.rename(columns={c: new_col}, inplace=True)

gtfs_routepoints = pd.read_csv("../data/gtfs_routepoints.csv", dtype=str)
for c in gtfs_routepoints.columns:
    new_col = "gtfs_" + c
    gtfs_routepoints.rename(columns={c: new_col}, inplace=True)

routepoints = osm_routepoints.merge(
                    gtfs_routepoints, how='inner', 
                    left_on=["osm_osm:ref:FR:STIF", "osm_osm:ref:FR:STIF:ExternalCode_Line"],
                    right_on=["gtfs_ZDEr_ID_REF_A", "gtfs_route_id"]
               )
routepoints.head()

,osm_stop_id,osm_name,osm_code,osm_destination,osm_network,osm_osm:ref:FR:STIF,osm_osm:ref:FR:STIF:ExternalCode_Line,osm_lat,osm_lon,gtfs_stop_id,gtfs_route_id,gtfs_ZDEr_ID_REF_A,gtfs_stop_name,gtfs_stop_lat,gtfs_stop_lon,gtfs_wheelchair_boarding,gtfs_dest_name,gtfs_route_short_name,gtfs_agency_name
0,StopPoint:Node:345242715,Cachin Blatreux,G'bus,Goussainville RER,Les Courriers de l'Île-de-France,38457,014014099:Gbus,49.0416155,2.4636214,StopPoint:8:30166,014014099:Gbus,38457,Cachin Blatreux,49.041729,2.46292,NaN,Raymond Lefevre,G'bus,CIF
1,StopPoint:Node:372723106,Carrefour de l'Europe,G'bus,Goussainville RER,Les Courriers de l'Île-de-France,38478,014014099:Gbus,49.021687,2.4653799,StopPoint:8:30167,014014099:Gbus,38478,Carrefour de l Europe,49.022577,2.465278,NaN,Raymond Lefevre,G'bus,CIF
2,StopPoint:Node:5320764716,Chapellerie,30B,Gare de Goussainville,Les Courriers de l'Île-de-France,38484,014014030:30B,49.0380727,2.4553955,StopPoint:8:30170,014014030:30B,38484,Chapellerie,49.038061,2.455352,NaN,Ampère/Chartrel,30B,CIF
3,StopPoint:Node:5320764716,Chapellerie,30B,Gare de Goussainville,Les Courriers de l'Île-de-France,38484,014014030:30B,49.0380727,2.4553955,StopPoint:8:30170,014014030:30B,38484,Chapellerie,49.038061,2.455352,NaN,Chapellerie,30B,CIF
4,StopPoint:Node:5320764716,Chapellerie,30B,Gare de Goussainville,Les Courriers de l'Île-de-France,38484,014014030:30B,49.0380727,2.4553955,StopPoint:8:30170,014014030:30B,38484,Chapellerie,49.038061,2.455352,NaN,Goussainville RER D,30B,CIF


In [3]:
routepoints_strict = routepoints[routepoints["osm_name"] == routepoints["gtfs_stop_name"]]
routepoints_strict = routepoints_strict[routepoints_strict["osm_destination"] == routepoints_strict["gtfs_dest_name"]]

len(routepoints_strict)

97

In [5]:
#from geopy.distance import vincenty
#dist = vincenty((row['stop_lat'], row['stop_lon']), (stop['stop_lat'], stop['stop_lon'])).meters
routepoints_strict

,osm_stop_id,osm_name,osm_code,osm_destination,osm_network,osm_osm:ref:FR:STIF,osm_osm:ref:FR:STIF:ExternalCode_Line,osm_lat,osm_lon,gtfs_stop_id,gtfs_route_id,gtfs_ZDEr_ID_REF_A,gtfs_stop_name,gtfs_stop_lat,gtfs_stop_lon,gtfs_wheelchair_boarding,gtfs_dest_name,gtfs_route_short_name,gtfs_agency_name
8954,StopPoint:Node:3140222572,Les Monrois,4,Gare RER Sucy-Bonneuil,SITUS,21408,040240004:4,48.7740601,2.5293818,StopPoint:35:1146,040240004:4,21408,Les Monrois,48.774157,2.52938,NaN,Gare RER Sucy-Bonneuil,4,SITUS
8960,StopPoint:Node:3140222569,Moulin de Touillon,4,Gare RER Sucy-Bonneuil,SITUS,21410,040240004:4,48.7765115,2.526554,StopPoint:35:1148,040240004:4,21410,Moulin de Touillon,48.776507,2.526533,NaN,Gare RER Sucy-Bonneuil,4,SITUS
9886,StopPoint:Node:4509237126,Centre Hospitalier,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416918,251195011:95-11,49.079394099999995,1.7598053999999999,StopPoint:87:66,251195011:95-11,416918,Centre Hospitalier,49.079438,1.759963,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
9895,StopPoint:Node:4509237126,Centre Hospitalier,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416918,251195011:95-11,49.079394099999995,1.7598053999999999,StopPoint:87:66,251195011:95-11,416918,Centre Hospitalier,49.079438,1.759963,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
10042,StopPoint:Node:2280210623,Mairie de Vetheuil,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416875,251195011:95-11,49.0630524,1.7016151,StopPoint:87:84,251195011:95-11,416875,Mairie de Vetheuil,49.06303,1.701562,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
10052,StopPoint:Node:2280210623,Mairie de Vetheuil,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416875,251195011:95-11,49.0630524,1.7016151,StopPoint:87:84,251195011:95-11,416875,Mairie de Vetheuil,49.06303,1.701562,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
10062,StopPoint:Node:4510738618,Route de Vétheuil,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416873,251195011:95-11,49.052963399999996,1.6968950999999999,StopPoint:87:85,251195011:95-11,416873,Route de Vétheuil,49.059608,1.700209,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
10072,StopPoint:Node:4510738618,Route de Vétheuil,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416873,251195011:95-11,49.052963399999996,1.6968950999999999,StopPoint:87:85,251195011:95-11,416873,Route de Vétheuil,49.059608,1.700209,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
10080,StopPoint:Node:4510740874,Mairie,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416890,251195011:95-11,49.040521899999995,1.6887538,StopPoint:87:86,251195011:95-11,416890,Mairie,49.039958,1.689065,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
10088,StopPoint:Node:4510740874,Mairie,95-11,Gare Mantes-la-Jolie,Busval d'Oise,416890,251195011:95-11,49.040521899999995,1.6887538,StopPoint:87:86,251195011:95-11,416890,Mairie,49.039958,1.689065,NaN,Gare Mantes-la-Jolie,95-11,Busval d'Oise
